## Generate baselines for UserC1 
This file needs to be executed in Windows as it uses win32 to generate Excel charts

In [1]:
import altair as alt
from altair_saver import save
import pydataset 
import itertools
import json
import random
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
import math
import toolz
import glob

import datetime
import win32com.client as win32
from win32com.client import Dispatch
from PIL import ImageGrab # for this module you should have "Download Windows x86-64 executable installer" You can download it from link:-https://www.python.org/downloads/windows/ 
import os

In [2]:
UserBatch = pd.read_csv(os.path.join('UserC1', 'human_batch_results.csv'))

In [3]:
def getEditTimes(d):
    return len(json.loads(json.loads(d)[0]['history']))
editTimes = [getEditTimes(d) for d in UserBatch['Answer.taskAnswers'].values]
pd.DataFrame({'time': UserBatch['WorkTimeInSeconds'].values, 'edit': editTimes})\
    .to_csv(os.path.join('UserC1', 'userc1HumanResult.csv'), index= False)

In [4]:
save_Folder = 'UserC1/ours'

In [5]:
## scoreRecord is the score given by our model
scoreRecord = pd.read_csv('UserC1/scoreRecord.csv')
scoreRecord.head()

,score,idx,nbar,width,bandwidth
0,0.003002,0,5,300,0.10
1,0.074736,1,5,300,0.15
2,0.207065,2,5,300,0.20
3,0.248151,3,5,300,0.25
4,0.350648,4,5,300,0.30


## Read Human baseline and Generate Charts

In [46]:
for jsonPath in glob.glob('UserC1/human/*.json'):
    with open(jsonPath) as f:
        vSpec = json.load(f)
    imgIdx = os.path.split(jsonPath)[1].split('.')[0]
    
    nBar = len(json.loads(vSpec['data']['values']))
        
    bestParas = scoreRecord[scoreRecord.nbar == nBar].sort_values(by='score', ascending = False).iloc[0]
    
    vSpec['width'] = bestParas['width']
    vSpec['encoding']['x']['band'] = bestParas['bandwidth']
    
    vSpec['encoding']['x']['title'] = None
    vSpec['encoding']['y']['title'] = None
    vSpec['encoding']['x']['scale']['paddingInner'] = 0
    
    chart = alt.Chart.from_dict(vSpec)
    
    namePrefix = imgIdx.split('_')[0] + '_2'
    chart.save(os.path.join(save_Folder, namePrefix + '.png'))
    with open(os.path.join(save_Folder, namePrefix + '.json'), 'w') as f:
        json.dump(vSpec, f)

In [51]:
save_Folder = 'UserC1/random'
for jsonPath in glob.glob('UserC1/result/*.json'):
    with open(jsonPath) as f:
        vSpec = json.load(f)
    imgIdx = os.path.split(jsonPath)[1].split('.')[0]
    
    nBar = len(json.loads(vSpec['data']['values']))
        
#     bestParas = scoreRecord[scoreRecord.nbar == nBar].sort_values(by='score', ascending = False).iloc[0]
    
    vSpec['width'] = random.sample(list(range(300,1200,100)), 1)[0]
    vSpec['encoding']['x']['band'] = random.sample(list(range(10,101,5)), 1)[0] / 100
    
    vSpec['encoding']['x']['title'] = None
    vSpec['encoding']['y']['title'] = None
    vSpec['encoding']['x']['scale']['paddingInner'] = 0
    
    chart = alt.Chart.from_dict(vSpec)
    
    namePrefix = imgIdx.split('_')[0] + '_3'
    chart.save(os.path.join(save_Folder, namePrefix + '.png'))
    with open(os.path.join(save_Folder, namePrefix + '.json'), 'w') as f:
        json.dump(vSpec, f)

## Generate Excel

In [ ]:
for filePath in glob.glob(os.path.join(folder, '*.json')):
    fileName = os.path.split(filePath)[1].split('.')[0]
    with open(filePath) as json_file:
        vSpec = json.load(json_file)
    data = json.loads(vSpec['data']['values'])
    
    excelName = fileName.split('_')[0] + '_1'
    bookpath = os.path.abspath('')+'/excelSheet/'+excelName+'.xlsx'
    workbook = xlsxwriter.Workbook(bookpath)
    worksheet = workbook.add_worksheet()
    worksheet.write('A1', 'EMAE')
    worksheet.write_column('A2', [x['NAME'] for x in data])
    worksheet.write('B1', 'hits86')
    worksheet.write_column('B2', [x["hits86"] for x in data])
        
    chart1 = workbook.add_chart({'type': 'column'})
    chart1.add_series({
        'name':'=Sheet1!$B$1',
        'categories':'=Sheet1!$A$2:$A$' + str(len(data) + 1),
        'values': '=Sheet1!$B$2:$B$' + str(len(data)+1),
        'fill':  {'color': '#3b6d9c'}
    })
    
    chart1.set_title({'none': True})
    chart1.set_legend({'none': True})
    chart1.set_x_axis({'num_font': {'size': 9, 'rotation': 0}, 'major_gridlines': {'visible': False}, 'major_tick_mark': 'none','minor_tick_mark': 'outside'})
    chart1.set_y_axis({'num_font': {'size': 9, 'rotation': 0}, 'major_gridlines': {'visible': False}, 'major_tick_mark': 'outside','minor_tick_mark': 'none'})
    chart1.set_chartarea({
        'border': {'none': True}
    })
    chart1.set_plotarea({
        'border': {'color': '#dddddd'}
    })

    worksheet.insert_chart('D2', chart1, {'x_offset': 25, 'y_offset': 10})                      
    workbook.close()

    xlApp = Dispatch('Excel.Application')
    # To open existing spread sheet.
    abspath = os.path.abspath('')+'/excelSheet/'+excelName+'.xlsx'
    workbook = xlApp.Workbooks.Open(abspath)
    #To select particular worksheet of the workbook by name.
    worksheet_name='sheet1'
    xlApp.Sheets(worksheet_name).Select()

    # To create an object for the worksheet.
    xlSheet1 = xlApp.Sheets(worksheet_name)

    #WARNING: The following line will cause the script to discard any unsaved changes in your workbook
    #Ensure to save any work before running script
    xlApp.DisplayAlerts = False
    
    currentChart = xlSheet1.ChartObjects()[0]
    currentChart.Copy
    figpath = os.path.abspath('')+'/excelFig/'+excelName+'.png'
    currentChart.Chart.Export(figpath)
    workbook.Close(True)